In [1]:
import json
import re
from tqdm import tqdm

In [2]:
with open("news.json") as f:
    news_list = json.load(f)

In [3]:
documents = []
invalid_line_patterns = re.compile(
    r"무단\s*전재|배포\s*금지|Copyrights|관련기사|기사\s*제보|여러분의 제보|카카오톡\s*:s*"
)

for article in tqdm(news_list, mininterval=1):
    text = article["text"]

    lines = text.split("\n")
    filtered_lines = []

    for line in lines:
        line = " ".join(line.split())

        # OOO 기자 패턴 제거
        line = re.sub(r"\w+ 기자", "", line)
        # email 주소 제거
        line = re.sub(r"\w+@\w+\.\w+", "", line)
        # 뉴스에서 []는 [논산] [대구=뉴시스] 등 다양한 참조를 의미하므로 제거
        line = re.sub(r"\[.*\]", "", line)
        # (서울=연합뉴스) 같은 패턴 제거
        line = re.sub(r"\(.*=.*\)", "", line)
        # 2024.10.29/뉴스1 같은 패턴 제거
        line = re.sub(r"\d{4}\.\d{2}\.\d{2}/.*\b", "", line)

        # 공백 제거
        line = " ".join(line.split())

        if invalid_line_patterns.search(line):
            # 무단전재, 배포금지 등이 포함된 문장 이후 문장들은 제외
            break

        # 한국어가 10자 이상 포함된 경우만 포함
        num_korean_chars = len(re.findall(r"[ㄱ-ㅎ가-힣]", line))
        if num_korean_chars >= 10:
            filtered_lines.append(line)

    text = "\n".join(filtered_lines)
    # 소문자로 변환
    text = text.lower()

    # 한국어가 50자 이상 포함된 경우만 포함
    num_korean_chars = len(re.findall(r"[ㄱ-ㅎ가-힣]", text))
    if num_korean_chars >= 50:
        documents.append(text)

# 중복 제거
documents = list(set(documents))

100%|██████████| 9604/9604 [00:02<00:00, 3901.78it/s]


In [ ]:
from kiwipiepy import Kiwi

kiwi = Kiwi() #load kiwi

document_tokens_list = []

for document in tqdm(documents, mininterval=1):
    #tokenize use kiwi
    tokens = [token.form for token in kiwi.tokenize(document)]

    #if token >= 10 put in doct list
    if len(tokens) >= 10:
        document_tokens_list.append((document, tokens))

100%|██████████| 7880/7880 [02:22<00:00, 55.26it/s]


In [ ]:
from gensim.models.word2vec import Word2Vec

dimension = 128 #convert word into 128 dimensional vector
#similar word = similar vector
#unsame word = far apart (negative words)

word2vec = Word2Vec(
    #use clean data from token to train
    sentences=[tokens for _, tokens in document_tokens_list],
    vector_size=dimension,
    min_count=10, #ignore word come less than 10. rare word are noise
    sg=1, # sg=1 = skipgram sg= 0 cbow
    workers=4,
)

In [ ]:
len(word2vec.wv.key_to_index) #see the word that got embedded. word:index

13351

In [ ]:
word2vec.wv["저출산"]  #show in 128 dimenstio matrix

array([-0.14436108, -0.6728425 ,  0.42717922, -0.02503245,  0.28129384,
       -0.5748861 , -0.13230786, -0.26044425,  0.28304553, -0.1035008 ,
        0.02134698, -0.25134763, -0.6525757 ,  0.29241303,  0.30718377,
        0.3168591 , -0.2690622 ,  0.6378406 ,  0.20351551,  0.13090998,
        0.47144797,  0.23624171,  0.15637122, -0.22894427,  0.34819803,
        0.53402597,  0.29751694, -0.07736859,  0.44035646,  0.04821702,
        0.12179998,  0.81935346,  0.0901124 ,  0.06763581, -0.32814705,
        0.7698681 ,  0.5220625 , -0.22950764, -0.09775116,  0.35656187,
        0.49254754,  0.38233575, -0.15546325, -0.18273942,  0.07226826,
        0.55556023, -0.10163607, -0.36923897, -0.7952047 , -0.06844834,
        0.10075079, -0.37676796,  0.13768177, -0.01207493, -0.06462228,
        0.17607294,  0.64535826, -0.5870449 , -0.5829941 ,  0.03188203,
        0.26531914,  0.07526039,  0.18578029, -0.25034085,  1.1055132 ,
        0.4380595 , -0.51028204,  0.04706103,  0.48990476, -0.13

In [ ]:
word2vec.wv.similar_by_word("저출산", 10) #print 10 similar by word 

[('우크라이나', 0.7159690260887146),
 ('벨라루스', 0.6924394965171814),
 ('푸틴', 0.6633460521697998),
 ('모스크바', 0.662726879119873),
 ('선희', 0.6476637721061707),
 ('오베르추크', 0.6438256502151489),
 ('최선희', 0.6383201479911804),
 ('침공', 0.6325005888938904),
 ('알렉세이', 0.6201255917549133),
 ('외무상', 0.6175360083580017)]

In [ ]:
word2vec.wv.similarity("남자", "여자") #check word similarity

0.6451614

In [ ]:
import numpy as np

w1 = word2vec.wv["남자"]
w2 = word2vec.wv["여자"]

print(np.dot(w1, w2) / (np.linalg.norm(w1) * np.linalg.norm(w2)))#check word similarity 
#but use manual log calc

0.64516145


In [ ]:
word2vec.wv.most_similar(positive=["서울", "수도권"], negative=["지방"], topn=20)
#get similar  20 word to positive avoid negative word.


[('한껏', 0.36258846521377563),
 ('걸맞', 0.34917300939559937),
 ('번창', 0.33775898814201355),
 ('국력', 0.3361835777759552),
 ('튼튼', 0.33169025182724),
 ('일류', 0.3180806338787079),
 ('우리', 0.31189751625061035),
 ('한류', 0.3095971345901489),
 ('우리나라', 0.3093096613883972),
 ('국운', 0.3077029287815094),
 ('저력', 0.3011007010936737),
 ('국격', 0.29771944880485535),
 ('절호', 0.2946532368659973),
 ('군대', 0.2889138460159302),
 ('국익', 0.28568434715270996),
 ('매개체', 0.2835228145122528),
 ('보유국', 0.28136005997657776),
 ('라며', 0.27619534730911255),
 ('눈부시', 0.27489134669303894),
 ('지평', 0.27459338307380676)]

In [23]:
word2vec.wv.most_similar(positive=["지방"], negative=["서울", "수도권"], topn=20)

[('확실히', 0.1707804948091507),
 ('성과', 0.15819254517555237),
 ('기회', 0.1513046771287918),
 ('튼튼', 0.1455494612455368),
 ('우선순위', 0.13711942732334137),
 ('자주국방', 0.13555872440338135),
 ('성취', 0.13308386504650116),
 ('매개체', 0.13031631708145142),
 ('세심', 0.12953375279903412),
 ('치밀', 0.12358049303293228),
 ('예우', 0.12189313769340515),
 ('결과물', 0.11718796193599701),
 ('겠', 0.11632491648197174),
 ('국익', 0.11589697748422623),
 ('튼튼하', 0.11494424194097519),
 ('응원', 0.11228194832801819),
 ('ᆸ시다', 0.1107255220413208),
 ('걸맞', 0.11027456074953079),
 ('고개', 0.1087050661444664),
 ('부합', 0.10795063525438309)]

In [ ]:
from gensim.models import FastText

dimension = 128 #dimen

fasttext = FastText(
    sentences=[tokens for _, tokens in document_tokens_list],
    vector_size=dimension,
    min_count=10, #< 10 ignore
    sg=1, #skipgram
    workers=4,
)


In [67]:
fasttext.wv.most_similar(positive=["서울", "수도권"], negative=["지방"], topn=20)

[('마포', 0.5779337286949158),
 ('포 시즌스', 0.5515439510345459),
 ('영등포구', 0.5414703488349915),
 ('서초동', 0.5329918265342712),
 ('영등포', 0.5272544026374817),
 ('부천', 0.5096818208694458),
 ('강북', 0.5036703944206238),
 ('성수동', 0.5022822022438049),
 ('삼성동', 0.501463770866394),
 ('마포구', 0.4974789321422577),
 ('서울역', 0.4964357018470764),
 ('둔촌동', 0.49513500928878784),
 ('아파트값', 0.4916444420814514),
 ('역삼동', 0.4903755486011505),
 ('롯데월드타워', 0.48988643288612366),
 ('한남동', 0.4877351224422455),
 ('노원구', 0.48681965470314026),
 ('시청역', 0.4854176938533783),
 ('페어몬트', 0.4853866994380951),
 ('청량리', 0.4812600612640381)]

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

dimension = 128

#train doc2vec - document embedding , larn one vector for 1 doc, 1 vector per doc
doc2vec = Doc2Vec(
    documents=[
        #create tag for each doc
        TaggedDocument(words=tokens, tags=[str(i)])
        for i, (_, tokens) in enumerate(document_tokens_list)
    ],
    vector_size=dimension,
    min_count=10,
    workers=4,
)

In [ ]:
doc2vec.wv.most_similar(positive=["강남구", "집값"])
#get similar word with positive word but in doc2vec

[('전셋값', 0.6177747845649719),
 ('송파구', 0.5656211376190186),
 ('노원구', 0.5509284734725952),
 ('성북', 0.525770366191864),
 ('성동구', 0.5174793004989624),
 ('땅값', 0.5143045783042908),
 ('동대문구', 0.510901153087616),
 ('938', 0.49799153208732605),
 ('성수동', 0.49796727299690247),
 ('중구', 0.494922935962677)]

In [74]:
doc2vec.wv.get_mean_vector(["강남구", "집값"])

array([-0.02801296, -0.19658181, -0.07889026,  0.01073649, -0.01298841,
        0.09010877,  0.03856221, -0.02263764, -0.08337904,  0.00778542,
        0.05301411,  0.02931999, -0.11233705, -0.00940534,  0.02868988,
       -0.05450509,  0.02084727, -0.01695624,  0.11336904,  0.06300487,
        0.01923266,  0.02674555, -0.08762868, -0.04431376,  0.09776855,
       -0.0275414 , -0.10705452, -0.02760827,  0.03714631, -0.1399316 ,
        0.0597698 ,  0.04151881,  0.0027904 , -0.0066614 , -0.00566008,
       -0.03267984,  0.08474678, -0.15980522, -0.0524998 ,  0.09186453,
       -0.04463796,  0.0184387 , -0.01826465,  0.03412673,  0.09043032,
       -0.05506077,  0.05100982, -0.0872625 , -0.07648489, -0.04177199,
       -0.01950743, -0.02489886,  0.00837776, -0.05121358, -0.06435177,
       -0.09231475,  0.09234323, -0.01025608, -0.03503404, -0.07719912,
       -0.10302018, -0.00104833, -0.02459497, -0.05337315,  0.01846555,
       -0.08057377,  0.09168082, -0.0095262 , -0.01199139,  0.00

In [ ]:
doc_indices = doc2vec.dv.similar_by_vector(
    doc2vec.wv.get_mean_vector(["강남구", "집값"])
)

#find similar doc to mean vector doc
for doc_index, similarity in doc_indices:
    #print doc to that index
    print(document_tokens_list[int(doc_index)][0][:200])
    print()

“최근 아파트 매매 거래량이 증가하고 있고 서울 강남 3구(강남·서초·송파)의 집값이 전고점의 95% 수준까지 올라오는 등 회복기에 들어선 것으로 판단됩니다. 서울의 경우 1990년대생들이 주목하는 뉴타운 재개발 단지를 중심으로 집값이 더 상승할 것으로 예상합니다. 다만 교통과 일자리, 학군 등에 따른 집값 양극화 현상은 점점 더 심화할 것으로 보여 상급지

윤석열 정부가 8·8 부동산 대책을 내놨다. 폭등하는 서울과 수도권 아파트 값·전세값을 잡기 위한 대책이다. 서울 주변 그린벨트를 풀어 주택 공급을 확대하고, 스트레스 dsr(총부채원리금상환비율) 금리 상향으로 수요를 억제함으로써 서울 집값을 내리게 하겠다는 게 주요 골자다. 대책이 나온 지도 한달 보름이 훌쩍 넘었다. 그럼에도 서울과 서울 인근 집값은 계

정부의 오락가락 정책에 하반기 집값 향뱡을 두고 전문가들의 전망이 엇갈리고 있다. 9월부터 스트레스 dsr(총부채원리금상환비율) 2단계가 시행되면 대출이 어려워져 집값이 잡힐 것이란 예상이 있다. 반면 하반기 금리 인하가 시작하면 집값이 본격 상승할 것이란 이들도 많다.
27일 부동산업계에 따르면 지난해 말부터 서울 아파트값 상승세가 계속되면서 서울 아파트

주택 가액 기준이 9억원 이하 제한
신생아 특례대출 신청이 출시 5개월 만에 5조8597억원의 대출 신청이 들어온 것으로 확인됐다. 서울보다는 경기, 인천의 대출 신청 건이 많았다. 가계 대출 확대를 부추기고 집값 상승을 자극할 수 있다는 우려가 나온다.
신생아 특례대출은 대출 신청일 기준으로 2년 이내에 출산·입양한 무주택 가구나 1주택 가구(대환대출)에

정부가 집값 불안을 잠재우기 위해 지난 8월 8일 정부서울청사에서 ‘제8차 부동산 관계 장관회의’를 열고 ‘국민 주거 안정을 위한 주택공급 확대 방안(8·8 주택공급 대책)’을 발표하며 총력전에 나섰지만, 8월 둘째 주 서울 아파트 가격은 5년 11개월 만에 최대 폭인 0.32%나 급격히 올라가는 등 부동산 시장의 움직임은 여전히 심상치 않다

In [ ]:
dv1 = doc2vec.wv.get_mean_vector(["강남구", "집값"])
#get mean vector for the word

#get mean by manual
dv2 = np.mean(
    [doc2vec.wv.get_vector(token, norm=True) for token in ["강남구", "집값"]],
    axis=0,
)

#compare = true since mean same
print(np.allclose(dv1, dv2))

True
